In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN_co.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
l_wfn = wfndata.l_wfn

In [5]:
print(np.sum(l_wfn[0].evc_gk.data[0] * np.conj(l_wfn[0].evc_gk.data[0])))
for i in range(9):
    imag = np.real(l_wfn[0].evc_gk.data[i])
    real = np.imag(l_wfn[0].evc_gk.data[i])

    l_wfn[0].evc_gk.data[i] = imag + 1j * real

print(np.sum((real + 1j * imag)*(real - 1j * imag)))

(1.0000000000000013-1.2357154238372941e-17j)
(1.0000000000000002-1.5775357844250837e-18j)


In [6]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 4728.64it/s]


In [7]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")
            print(epsref.shape)
            print(epsinv.shape)

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 1/1 [00:00<00:00, 328.24it/s]

Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0
(15, 15)
(15, 15)


In [8]:
from epsilonv2 import Epsilonv2

epsclassv2 = Epsilonv2.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 5825.42it/s]


In [9]:
# Construct the Charge Matrix Class
from mtxel import ChargeMtxEL

cmtxclass = ChargeMtxEL.from_BGW(
    wfndata = wfndata,
    epsinp = epsinp,
    sigmainp = sigmainp,
    num_bands_val = 2,
    num_bands_con = 1,
)

In [10]:
mvc = cmtxclass.mvc(0)
print(f"Shape of mvc is {mvc.shape}")

mccp = cmtxclass.mccp(0, 0)
print(f"Shape of mccp is {mccp.shape}")

mvvp = cmtxclass.mvvp(0, 0)
print(f"Shape of mvvp is {mvvp.shape}")

Shape of mvc is (2, 1, 27)
Shape of mccp is (1, 1, 27)
Shape of mvvp is (2, 2, 27)


In [11]:
from read_mtxel import *
bgw_arrays = parse_file(dirname + "kernel.out")

mccp_BGW = bgw_arrays.get('mccp', None)
mvvp_BGW = bgw_arrays.get('mvvp', None)
mvc_BGW = bgw_arrays.get('mvc', None)
mvpcp_BGW = bgw_arrays.get('mvpcp', None)

In [12]:
print(f"Shape of mccp_BGW is {mccp_BGW.shape}")
print(f"Shape of mvvp_BGW is {mvvp_BGW.shape}")
print(f"Shape of mvc_BGW is {mvc_BGW.shape}")
print(f"Shape of mvpcp_BGW is {mvpcp_BGW.shape}")

Shape of mccp_BGW is (27, 1, 1, 1)
Shape of mvvp_BGW is (27, 1, 1, 1)
Shape of mvc_BGW is (27, 1, 1, 1)
Shape of mvpcp_BGW is (27, 1, 1, 1)


In [13]:
is_equal_mvc_mvpcp = np.allclose(mvc_BGW, mvpcp_BGW)
print(f"Is mvc and mvpcp equal? {is_equal_mvc_mvpcp}")

Is mvc and mvpcp equal? True


In [14]:
mccp = mccp[0, 0, :]
mccp_BGW = mccp_BGW[:, 0, 0, 0]

for i in range(27):
    mccp_value = mccp[i] if i < len(mccp) else 0
    print(f"mccp[{i}]: {mccp_value.real:.10f}{mccp_value.imag:+.10f}j, mccp_BGW[{i}]: {mccp_BGW[i].real:.10f}{mccp_BGW[i].imag:+.10f}j")

mccp[0]: 1.0000000000-0.0000000000j, mccp_BGW[0]: 1.0000000000+0.0000000000j
mccp[1]: -0.1069011136+0.0000004958j, mccp_BGW[1]: -0.1069011136+0.0000004958j
mccp[2]: 0.1071866199+0.0000009394j, mccp_BGW[2]: 0.1071866199+0.0000009394j
mccp[3]: 0.1071902943+0.0000008143j, mccp_BGW[3]: 0.1071902943+0.0000008143j
mccp[4]: 0.1071935416+0.0000005033j, mccp_BGW[4]: 0.1071935416+0.0000005033j
mccp[5]: 0.1071935416-0.0000005033j, mccp_BGW[5]: 0.1071935416-0.0000005033j
mccp[6]: 0.1071902943-0.0000008143j, mccp_BGW[6]: 0.1071902943-0.0000008143j
mccp[7]: 0.1071866199-0.0000009394j, mccp_BGW[7]: 0.1071866199-0.0000009394j
mccp[8]: -0.1069011136-0.0000004958j, mccp_BGW[8]: -0.1069011136-0.0000004958j
mccp[9]: -0.3178784615+0.0000016715j, mccp_BGW[9]: -0.3178784615+0.0000016715j
mccp[10]: -0.3182568800+0.0000008144j, mccp_BGW[10]: -0.3182568800+0.0000008144j
mccp[11]: -0.3146274045+0.0000020659j, mccp_BGW[11]: -0.3146274045+0.0000020659j
mccp[12]: -0.3146274045-0.0000020659j, mccp_BGW[12]: -0.314627

In [15]:
mvc = mvc[0, 0, :]
mvc_BGW = mvc_BGW[:, 0, 0, 0]

for i in range(27):
    mvc_value = mvc[i] if i < len(mvc) else 0
    print(f"mvc[{i}]: {mvc_value.real:.4f}{mvc_value.imag:+.4f}j, mvc_BGW[{i}]: {mvc_BGW[i].real:.4f}{mvc_BGW[i].imag:+.4f}j")

mvc[0]: -0.0000+0.0000j, mvc_BGW[0]: -0.0000-0.0000j
mvc[1]: 0.0547-0.0249j, mvc_BGW[1]: -0.0547-0.0249j
mvc[2]: -0.2047+0.0932j, mvc_BGW[2]: 0.2047+0.0932j
mvc[3]: -0.2039+0.0928j, mvc_BGW[3]: 0.2039+0.0928j
mvc[4]: -0.2035+0.0926j, mvc_BGW[4]: 0.2035+0.0926j
mvc[5]: 0.2035-0.0926j, mvc_BGW[5]: -0.2035-0.0926j
mvc[6]: 0.2039-0.0928j, mvc_BGW[6]: -0.2039-0.0928j
mvc[7]: 0.2047-0.0932j, mvc_BGW[7]: -0.2047-0.0932j
mvc[8]: -0.0547+0.0249j, mvc_BGW[8]: 0.0547+0.0249j
mvc[9]: -0.1620+0.0738j, mvc_BGW[9]: 0.1620+0.0738j
mvc[10]: -0.1605+0.0731j, mvc_BGW[10]: 0.1605+0.0731j
mvc[11]: -0.1570+0.0715j, mvc_BGW[11]: 0.1570+0.0715j
mvc[12]: 0.1570-0.0715j, mvc_BGW[12]: -0.1570-0.0715j
mvc[13]: 0.1605-0.0731j, mvc_BGW[13]: -0.1605-0.0731j
mvc[14]: 0.1620-0.0738j, mvc_BGW[14]: -0.1620-0.0738j
mvc[15]: -0.0064+0.0029j, mvc_BGW[15]: 0.0064+0.0029j
mvc[16]: -0.0045+0.0021j, mvc_BGW[16]: 0.0045+0.0021j
mvc[17]: -0.0061+0.0028j, mvc_BGW[17]: 0.0061+0.0028j
mvc[18]: -0.0001+0.0000j, mvc_BGW[18]: 0.0001+0

In [16]:
mvvp = mvvp[0, 0, :]
mvvp_BGW = mvvp_BGW[:, 0, 0, 0]

for i in range(27):
    mvvp_value = mvvp[i] if i < len(mvvp) else 0
    print(f"mvvp[{i}]: {mvvp_value.real:.4f}{mvvp_value.imag:+.4f}j, mvvp_BGW[{i}]: {mvvp_BGW[i].real:.4f}{mvvp_BGW[i].imag:+.4f}j")

mvvp[0]: 1.0000-0.0000j, mvvp_BGW[0]: 1.0000+0.0000j
mvvp[1]: 0.2551+0.0000j, mvvp_BGW[1]: 0.2551+0.0000j
mvvp[2]: 0.4264-0.0000j, mvvp_BGW[2]: 0.4264-0.0000j
mvvp[3]: 0.4259-0.0000j, mvvp_BGW[3]: 0.4259-0.0000j
mvvp[4]: 0.4235-0.0000j, mvvp_BGW[4]: 0.4235-0.0000j
mvvp[5]: 0.4235+0.0000j, mvvp_BGW[5]: 0.4235+0.0000j
mvvp[6]: 0.4259+0.0000j, mvvp_BGW[6]: 0.4259+0.0000j
mvvp[7]: 0.4264+0.0000j, mvvp_BGW[7]: 0.4264+0.0000j
mvvp[8]: 0.2551-0.0000j, mvvp_BGW[8]: 0.2551-0.0000j
mvvp[9]: 0.2976-0.0000j, mvvp_BGW[9]: 0.2976-0.0000j
mvvp[10]: 0.2955-0.0000j, mvvp_BGW[10]: 0.2955-0.0000j
mvvp[11]: 0.2951-0.0000j, mvvp_BGW[11]: 0.2951-0.0000j
mvvp[12]: 0.2951+0.0000j, mvvp_BGW[12]: 0.2951+0.0000j
mvvp[13]: 0.2955+0.0000j, mvvp_BGW[13]: 0.2955+0.0000j
mvvp[14]: 0.2976+0.0000j, mvvp_BGW[14]: 0.2976+0.0000j
mvvp[15]: 0.2722-0.0000j, mvvp_BGW[15]: 0.2722-0.0000j
mvvp[16]: 0.2729-0.0000j, mvvp_BGW[16]: 0.2729-0.0000j
mvvp[17]: 0.2761-0.0000j, mvvp_BGW[17]: 0.2761-0.0000j
mvvp[18]: 0.1545-0.0000j, mvvp